# Problem Set 2: Local & Adversarial Search

**Release Date:** 2 September 2024

**Due Date:** 23:59, 14 September 2024

## Overview

In class, we discussed a range of search algorithms. In this problem set, we will get some hands-on practice by implementing local search for the **Travelling Salesman Problem (TSP)**, which operates in a fully-observable, single-agent, deterministic, episodic, static, and discrete environment. We will also get some hands-on practice on Adversarial Search by coding an AI to play the game **Breakthrough**.

**Required Files**:
* ps2.py
* breakthrough.py

**Plagiarism Policy**: Please refer to our [Course Policies](https://canvas.nus.edu.sg/courses/62323/pages/course-policies)

**IMPORTANT**: While it is possible to write and run Python code directly in Jupyter notebook, we recommend that you do this Problem set with an IDE using the .py file provided. An IDE will make debugging significantly easier.

**Post-Problem Set Survey**:
Your feedback is important to us! After completing Problem Set 2, please take a moment to share your thoughts by filling out this [survey](https://coursemology.org/courses/2851/surveys/2387).

## The Travelling Salesman Problem (TSP)

Your cousin Ben Bitdiddle is planning to start a company which sells premium imported chocolate from Europe. Among all cities in the country, Ben must choose one to be his company headquarters to receive shipment from Europe and devise a route from the headquarters to deliver the chocolate to every other city. This route must only visit each city **exactly once** and return to the headquarters to receive the next shipment. In addition, to save fuel cost, the route must be **as short as possible**. Given a list of cities and the distance between every two cities, what is the shortest possible route?

This problem is a classic NP-hard optimisation problem in computer science. In this task, you will design and implement a local search algorithm to find a shortest route. You must find the route as **a list of cities** in the order of travel from the starting city to the last city before returning.

For example, consider the graph below, which represents 4 cities and the distances between them.

<pre>
<p style="text-align: center;">
<img src="images/tsp.png", width = 400>
</pre>

An optimal route is `[0, 1, 2, 3]`, with the minimal distance travelled of 1 + 2 + 2 + 3 = 8.

**Note:**
* There can be more than 1 shortest route, e.g., `[1, 0, 3, 2]`, `[1, 3, 2, 0]`, etc. You only need to find one such route.
* `[0, 1, 2]` is not legal as the route must go through all 4 cities.
* `[0, 1, 2, 3, 1]` is not legal as city 1 is visited more than once.
* `[1, 3, 0, 2]` is legal but it is not the shortest route, as the distance travelled of 3 + 3 + 2 + 2 = 10.

### Task 1.1: State representation
Propose a state representation for this problem if we want to formulate it as a local search problem.

The state representation will be a list of cities in the order of travel consisiting of initial city to last city(before return to initial city).For example, `[0, 1, 2, 3]` represents the route that starts from city 0, then goes to city 1, then city 2, and finally city 3 before returning to city 0.

### Task 1.2: Initial and goal states

What are the initial and goal states for the problem under your proposed representation?

The initial state is a random permuation of the cities to create a initial route.(using the import Random) e.g [1,3,2,0]

The goal state is the shortest route that visits each city and returns to the starting city. e.g [0,1,2,3]

**Note:**
* In many optimization problems such as the TSP, the path to the goal is irrelevant; **the goal state itself is the solution to the problem**.
* Local search algorithms keep a single "current" state and move from a state to another in the search space by applying local changes (with the help of a *transition function*), until an optimal solution is found.

### Can you do better?

Recall that similar to A* search, local search utilises evaluation functions to decide how to transition from one state to another. However, being an uninformed guy, your cousin Ben Bitdiddle tells you to use the "greedy" solution. Given an incomplete route, the "greedy" solution builds a path by adding the closest unvisited node from the last visited node, until all nodes are visited. For instance, in the graph above, the "greedy" solution is `[0, 1, 2, 3]`.

Although this solution seems relatively sensible, as a CS2109S student, you have a nagging feeling that it may not work all the time. Can you create an evaluation function and transition function to get better results with local search?


**Note:**

* For the following tasks, we will be benchmarking your hill-climbing algorithm against our own version using the greedy solution. Note that the hidden test cases can be quite large, so any brute-force solution will not suffice. 

* Your own evaluation functions and transition functions may underperform against the greedy solution for small instances of TSP, but should outperform the greedy solution consistently for large instances. For our public and private test cases, we have designed the greedy solution to be suboptimal.

* If your code does not pass the private test cases on Coursemology because it underperforms against the greedy solution, you may re-run your code a few times in case you are "unlucky" with random initial routes.

### Task 1.3: State transitions

Implement a reasonable transition function `transition(route)` to generate new routes by applying minor "tweaks" to the current route. It should return a list of new routes to be used in the next iteration in the hill-climbing algorithm.

**Note:**
* At each iteration, the routes generated from the transition function are evaluated against each other (using an evaluation function). The best route will be selected for the next iteration if it is better than the current route.
* Your transition function should not return too many routes as it would take too much time for evaluation. (do not enumerate all possible states otherwise it will timeout, only generate "neighbors")
* However, if too few routes are generated, you are more likely to be stuck at a local maxima as each route will be compared against fewer routes.

Please run the following cell before proceeding. You may use any of the imported libraries/classes here.

In [2]:
"""
Run this cell before you start!
"""

import random
import time

from typing import List, Tuple, Callable

In [3]:
def transition(route: List[int]) -> List[List[int]]:
    """
    Generates new routes to be used in the next iteration in the hill-climbing algorithm.

    Args:
        route (List[int]): The current route as a list of cities in the order of travel.

    Returns:
        new_routes (List[List[int]]): New routes to be considered.
    """
    """ YOUR CODE HERE """
    new_routes = []
    #permuatation without repetition but do not include all
    for i in range(len(route)):
        for j in range(i+1,len(route)):
            new_route = route.copy() #prevent antialiasing , dun need deepcopy.copy() as it is not a list of list
            new_route[i],new_route[j] = new_route[j], new_route[i] #swap with all subsequent numbers after i
            new_routes.append(new_route)
    return new_routes
    # raise NotImplementedError
    """ YOUR CODE END HERE """

# Test cases
def test_transition(route):
    sorted_route = sorted(route)
    result = transition(route)
    assert result is not None, "Transition function returns an empty list."
    assert any(result), "Transition function returns an empty list."
    for new_route in result:
        assert len(new_route) == len(sorted_route), "New route does not have the same number of cities as the original route."
        assert sorted(new_route) == sorted_route, "New route does not contain all cities present in the original route."

permutation_route = [0, 1, 2, 3, 4]
new_permutation_routes = transition(permutation_route)
assert len(new_permutation_routes) < 24, "Your transition function may have generated too many new routes by enumerating all possible states."

test_transition([1, 3, 2, 0])
test_transition([7, 8, 6, 3, 5, 4, 9, 2, 0, 1])

### Task 1.4: Evaluation function
Implement an evaluation function `evaluation_func(cities, distances, route)` that would be helpful in deciding on the "goodness" of a route, i.e. an optimal route should return a higher evaluation score than a suboptimal one.

In [4]:


def evaluation_func(
    cities: int,
    distances: List[Tuple[int]],
    route: List[int]
) -> float:
    """
    Computes the evaluation score of a route

    Args:
        cities (int): The number of cities to be visited.

        distances (List[Tuple[int]]): The list of distances between every two cities. Each distance
            is represented as a tuple in the form of (c1, c2, d), where c1 and c2 are the two cities
            and d is the distance between them. The length of the list should be equal to cities *
            (cities - 1)/2.

        route (List[int]): The current route as a list of cities in the order of travel.

    Returns:
        h_n (float): the evaluation score.
    """
    """ YOUR CODE HERE """
    #generate all edge and compare with distance if it is add to distance value
    d = 0
    for i in range(cities):
        #note that the distances 0 - 1 will exist for 1 - 0 according the TSP problem, so need to check both ways
        #if it is the last city, return to the first city
        if i == len(route)-1:
            fro = route[i]
            to = route[0]
        else:
            fro = route[i]
            to = route[i+1]
        for edge in distances:
            if (edge[0] == fro and edge[1] == to) or (edge[1] == fro and edge[0] == to): 
                d += edge[2]
    #since goodness is determine by higher value and optimal path is smaller = better, use division of number of edges with path cost to get higher value 
    h_n = (cities*(cities-1)/2) / d
    return h_n
    # raise NotImplementedError
    """ YOUR CODE END HERE """

# Test cases
cities = 4
distances = [(1, 0, 10), (0, 3, 22), (2, 1, 8), (2, 3, 30), (1, 3, 25), (0, 2, 15)]

route_1 = evaluation_func(cities, distances, [0, 1, 2, 3])
route_2 = evaluation_func(cities, distances, [2, 1, 3, 0])
route_3 = evaluation_func(cities, distances, [1, 3, 2, 0])

assert route_1 == route_2
assert route_1 > route_3

### Task 1.5: Explain your evaluation function

Explain why your evaluation function is suitable for this problem. 

In my implementation, I used the total number of edges divided by the total distance travelled as the evaluation function. This is because the total number of edges is constant for a given number of cities, and the optimal route will always have the lower distance travelled than a suboptimal one. Therefore, this way, an optimal route with a smaller total distance travelled will have a higher evaluation score (the total distance travelled is inversely proportional to the evaluation score).

### Task 1.6: Implement hill-climbing
Using your representation above, implement the hill-climbing algorithm `hill_climbing(cities, distances, transition, evaluation_func)`, which takes in the number of cities and the list of distances, a transition function, an evaluation function and returns the shortest route as a list of cities.

1. The hill-climbing approach is a local search algorithm which starts with a randomly-initialised state and continuously selects the next candidate solution that locally maximizes the reduction of the evaluation function.

2. The algorithm terminates when a (local) maxima is reached, i.e. a solution that cannot be improved further by looking at the next candidate solutions.

3. Unlike previous search algorithms you have implemented, hill-climbing only keeps a single current state. As such, it does not involve a search tree/graph. Backtracking is also not possible.

Coursemology will tests this question with correct implementation of `transition(route)` and `evaluation_func(cities, distances, route)`, in case you were unable to come up with a good evaluation function and transition function. Note that the implemented `evaluation_func(cities, distances, route)` returns a float, which can be from `float(-inf)` to `float(inf)`. Locally, you can test your hill-climbing implementation using the functions you defined in Task 2.3.

If you are certain that your solution is correct but the test cases fail, just rerun your code

In [5]:
def hill_climbing(
    cities: int,
    distances: List[Tuple[int]],
    transition: Callable,
    evaluation_func: Callable
) -> List[int]:
    """
    Hill climbing finds the solution to reach the goal from the initial.

    Args:
        cities (int): The number of cities to be visited.

        distances (List[Tuple[int]]): The list of distances between every two cities. Each distance
            is represented as a tuple in the form of (c1, c2, d), where c1 and c2 are the two cities
            and d is the distance between them. The length of the list should be equal to cities *
            (cities - 1)/2.

        transition (Callable): A function that generates new routes to be used in the next
            iteration in the hill-climbing algorithm. Will be provided on Coursemology.

        evaluation_func (Callable): A function that computes the evaluation score of a route. Will
            be provided on Coursemology.

    Returns:
        route (List[int]): The shortest route, represented by a list of cities in the order to be
            traversed.
    """       
    """ YOUR CODE HERE """
    #create an initial route randomly
    current = random.sample(range(cities),cities)
    while True:
        #find next states,store current and current score
        current_unchanged = True
        current_score = evaluation_func(cities,distances,current) 
        neighbours = transition(current) #possible next states
        
        #check if neighbours have better score, if yes, replace
        for neighbour in neighbours:
            neighbour_score = evaluation_func(cities,distances,neighbour)
            if neighbour_score > current_score:
                current_unchanged = False
                current = neighbour
                current_score = neighbour_score
        
        #if not, optimal route found return route
        if current_unchanged:
            return current      
    raise NotImplementedError
    """ YOUR CODE END HERE """

# Test cases
def test_hill_climbing(cities: int, distances: List[Tuple[int]], transition: callable, evaluation_func: callable):
    route = hill_climbing(cities, distances, transition, evaluation_func)
    assert sorted(route) == list(range(cities)), "New route does not contain all cities present in the original route."

cities_1 = 4
distances_1 = [(1, 0, 10), (0, 3, 22), (2, 1, 8), (2, 3, 30), (1, 3, 25), (0, 2, 15)]

test_hill_climbing(cities_1, distances_1, transition, evaluation_func)

cities_2 = 10
distances_2 = [(2, 7, 60), (1, 6, 20), (5, 4, 70), (9, 8, 90), (3, 7, 54), (2, 5, 61),
    (4, 1, 106), (0, 6, 51), (3, 1, 45), (0, 5, 86), (9, 2, 73), (8, 4, 14), (0, 1, 51),
    (9, 7, 22), (3, 2, 22), (8, 1, 120), (5, 7, 92), (5, 6, 60), (6, 2, 10), (8, 3, 78),
    (9, 6, 82), (0, 2, 41), (2, 8, 99), (7, 8, 71), (0, 9, 32), (4, 0, 73), (0, 3, 42),
    (9, 1, 80), (4, 2, 85), (5, 9, 113), (3, 6, 28), (5, 8, 81), (3, 9, 72), (9, 4, 81),
    (5, 3, 45), (7, 4, 60), (6, 8, 106), (0, 8, 85), (4, 6, 92), (7, 6, 70), (7, 0, 22),
    (7, 1, 73), (4, 3, 64), (5, 1, 80), (2, 1, 22)]

test_hill_climbing(cities_2, distances_2, transition, evaluation_func)

### Task 1.7: Improve hill-climbing with random restarts

When no "better" neighbouring solutions are present, local search can be stuck at a local maxima. One way to combat this is to simply repeat local search from random initial states, taking the best performing iteration. 

Implement `hill_climbing_with_random_restarts(cities, distances, tansition, evaluation_func, hill_climbing, repeats)` by repeating hill climbing at different random locations.

* Coursemology will tests this question with correct implementation of `transition(route)`, `evaluation_func(cities, distances, route)` and `hill_climbing(cities, distances, transition, evaluation_func)`
* Note that the implemented `evaluation_func(cities, distances, route)` returns a float, which can be from `float(-inf)` to `float(inf)`.

In [6]:
from hmac import new
from turtle import distance


def hill_climbing_with_random_restarts(
    cities: int,
    distances: List[Tuple[int]],
    transition: Callable,
    evaluation_func: Callable,
    hill_climbing: Callable,
    repeats: int = 10
) -> List[int]:
    """
    Hill climbing with random restarts finds the solution to reach the goal from the initial.

    Args:
        cities (int): The number of cities to be visited.

        distances (List[Tuple[int]]): The list of distances between every two cities. Each distance
            is represented as a tuple in the form of (c1, c2, d), where c1 and c2 are the two cities
            and d is the distance between them. The length of the list should be equal to cities *
            (cities - 1)/2.

        transition (Callable): The transition function to be used in hill climbing. Will be
            provided on Coursemology.

        evaluation_func (Callable): The evaluation function to be used in hill climbing. Will be
            provided on Coursemology.

        hill_climbing (Callable): The hill climbing function to be used for each restart. Will be
            provided on Coursemology.

        repeats (int): The number of times hill climbing to be repeated. The default value is 10.

    Returns:
        route (List[int]): The shortest route, represented by a list of cities in the order to be
            traversed.
    """
    """ YOUR CODE HERE """
    route = random.sample(range(cities),cities)
    score = evaluation_func(cities,distances,route)
    #repeat hill climbing and check if score changes, if yes, replace
    for _ in range(repeats-1):
        new_route = hill_climbing(cities,distances,transition,evaluation_func)
        new_score = evaluation_func(cities,distances,new_route)
        # print(new_score)
        if new_score > score:
            route = new_route
            score = new_score
    return route
    raise NotImplementedError
    """ YOUR CODE END HERE """

# Test cases
def test_random_restarts(cities: int, distances: List[Tuple[int]], transition, evaluation_func, hill_climbing, hill_climbing_with_random_restarts, repeats: int = 10):
    route = hill_climbing_with_random_restarts(cities, distances, transition, evaluation_func, hill_climbing, repeats)
    assert sorted(route) == list(range(cities)), "New route does not contain all cities present in the original route."

cities_1 = 4
distances_1 = [(1, 0, 10), (0, 3, 22), (2, 1, 8), (2, 3, 30), (1, 3, 25), (0, 2, 15)]

test_random_restarts(cities_1, distances_1, transition, evaluation_func, hill_climbing, hill_climbing_with_random_restarts)

cities_2 = 10
distances_2 = [(2, 7, 60), (1, 6, 20), (5, 4, 70), (9, 8, 90), (3, 7, 54), (2, 5, 61),
    (4, 1, 106), (0, 6, 51), (3, 1, 45), (0, 5, 86), (9, 2, 73), (8, 4, 14), (0, 1, 51),
    (9, 7, 22), (3, 2, 22), (8, 1, 120), (5, 7, 92), (5, 6, 60), (6, 2, 10), (8, 3, 78),
    (9, 6, 82), (0, 2, 41), (2, 8, 99), (7, 8, 71), (0, 9, 32), (4, 0, 73), (0, 3, 42),
    (9, 1, 80), (4, 2, 85), (5, 9, 113), (3, 6, 28), (5, 8, 81), (3, 9, 72), (9, 4, 81),
    (5, 3, 45), (7, 4, 60), (6, 8, 106), (0, 8, 85), (4, 6, 92), (7, 6, 70), (7, 0, 22),
    (7, 1, 73), (4, 3, 64), (5, 1, 80), (2, 1, 22)]

test_random_restarts(cities_2, distances_2, transition, evaluation_func, hill_climbing, hill_climbing_with_random_restarts)

### Task 1.8: Comparison between local search and other search algorithms

Compared to previous search algorithms you have seen (uninformed search, A* search), why do you think local search is more suitable for this problem?

Local search is more suitable for this problem as it is a optimization problem where the path to the goal is irrelevant and goal can be any state. Uninformed search and A* search are not suitable for this problem as they are used to find the path to the goal state, and will have to search through all possible states to find the optimal solution without a goal state. Furthermore, as the number of cities increases, the number of possible states increases factorially, making it inefficient to use uninformed search or A* search. Local search, on the other hand, can still be used to find the optimal solution since it only requires the current state and the next state to evaluate. A* also requires a heuristic function which is not suitable for this problem since the goal state is not known.

## Breakthrough

Breakthrough was the winner of the 2001 8 × 8 Game Design Competition, sponsored by *About.com* and *Abstract Games Magazine*. When Dan Troyka formulated it, it was originally for a 7×7 board. We’re going to play it on a 6×6 board to limit the complexity. In terms of our terminology for the agent environment, Breakthrough is a fully observable, strategic, deterministic game. The game always results in a win for one of the two players.

How exactly do you design an agent to play this game and, most importantly, win? An agent takes sensory input and reasons about it, and then outputs an action at each time step. You thus need to create a program that can read in a representation of the board (that’s the input) and output a legal move in Breakthrough. You then need an evaluation function to evaluate how good a position is.

In this problem set, you will first implement a minimax agent, followed by augmenting it with alpha-beta pruning. You will then implement an improved evaluation function.

## Breakthrough Technical Description

<pre>
<p style="text-align: center;">
<img src="images/breakthrough_board.png">
Figure 1. Game Board
</p>
</pre>

*Figure 1* shows the starting position of our game board. The player controlling the black pieces can move the black pawns (**B**), and the other player controlling the white pieces can move the white pawns (**W**). Each player can only move pieces of their own colour during their turn. Black can move its pawns forward towards the bottom of the board, while white can move its pawns forward towards the top of the board. Black wins by moving any piece to the opposite side, row (horizontal) index 5. White wins by moving any piece to row (horizontal) index 0. A side also wins if their opponent has no pieces left. **Kindly follow the same indexing as provided in *Figure 1***.

<pre>
<p style="text-align: center;">
<img src="images/game_move_black.png">
Figure 2. Possible Moves for Black
</p>
</pre>

Pieces move one space directly forward or diagonally forward, and only capture diagonally forward. An example of possible moves for black is illustrated in *Figure 2*. In this figure, the black pawn at (3, 2) can move forward to any of the three spaces in row index 4. The black pawn at (0, 4) can either move diagonally right to (1, 5), or capture by moving diagonally left to (1, 3). It cannot capture by moving forward; its forward movement is blocked by the white pawn at (1, 4). Note that your move is not allowed to take your pawn outside the board.

<pre>
<p style="text-align: center;">
<img src="images/game_move_white.png">
Figure 3. Possible Moves for White
</p>
</pre>

The same movement rules apply to white as illustrated in *Figure 3*. In this figure, the white pawn at (1, 3) can move forward to any of the three spaces in row index 0. The white pawn at (4, 2) can either move diagonally right to (3, 3), or capture by moving diagonally left to (3, 1). It cannot capture by moving forward; its forward movement is blocked by the black pawn at (3, 2).

## Provided Utility Functions

You can use the functions provided in *breakthrough.py* file as you see fit. These functions have mainly been used by the game playing framework to facilitate the two player game. A short description of these functions is given below:

- `Player.get_opponent()`: Given a player colour, it returns the opponent's colour.
- `print_state(board)`: It takes in the board 2D list as parameter and prints out the current state of the board in a comprehensible way.
- `is_valid_move(board, src, dst, current_player)`: It takes in the board configuration, move source, move destination and current player colour as its parameters. It returns `True` if the move is valid and returns `False` if the move is invalid.
- `make_move(curr_board, src, dst, current_player, in_place=True)`: Given a board configuration, move source and move destination and current player colour, this function updates the board configuration based on the indicated move. This function updates the board configuration by modifying existing values if `in_place` is set to `True`, or creating a new board with updated values if `in_place` is set to `False`.
- `is_game_over(board)`: Given a board configuration, it returns `True` if the game is over, `False` otherwise.

Please run the following cell before proceeding. You may use any of the imported libraries/classes here. 

In [7]:
"""
Run this cell before you start!
"""

import copy
from typing import Callable, Union

import breakthrough

Score = Union[int, float]
Move = tuple[tuple[int, int], tuple[int, int]]
Board = list[list[str]]

To build your own agent, you will need a heuristic function to evaluate a position. The heuristic function below evaluates the board from black's perspective. You will be using this heuristic function for tasks 3 and 4.

In [8]:
def evaluate(board: Board) -> Score:
    """
    Returns the score of the current position.

    Parameters
    ----------
    board: 2D list of lists. Contains characters "B", "W", and "_",
    representing black pawn, white pawn, and empty cell, respectively.

    Returns
    -------
    An evaluation (as a Score).
    """
    bcount = 0
    wcount = 0
    for r, row in enumerate(board): #creates a list of tuples with index and value
        for tile in row: #iterate through each tile in the row
            if tile == "B": #if tile is black, check if it is at the end of the board
                if r == 5: 
                    return breakthrough.WIN #if it is, return the win score
                bcount += 1
            elif tile == "W":
                if r == 0:
                    return -breakthrough.WIN
                wcount += 1
    if wcount == 0:
        return breakthrough.WIN
    if bcount == 0:
        return -breakthrough.WIN
    return bcount - wcount
#basically counts the number of tiles of each color and returns the difference
#if hit other end of board, return win score

The provided heuristic function returns `breakthrough.WIN` if black wins, and `-breakthrough.WIN` if white wins. Otherwise, it takes the difference between the number of black pieces and the number of white pieces that are on the board. The value of `breakthrough.WIN` can be found in `breakthrough.py` and has a value of `21092109`.

**Note**: On Coursemology, we will provide and use this heuristic function to test your code in task 3 and task 4.

### Task 2.1: Implement a function to generate all valid moves

It is useful to generate all the possible moves that the current player can make in a certain position. You may need this function when implementing the minimax algorithm.

Implement `generate_valid_moves(board, current_player)` to generate all possible moves for a player given the current board state. The function should work for the current player to move, whether it is black or white.

**Note**: On Coursemology, we will provide you with the correct implementation of `generate_valid_moves` in task 3 and task 4.

In [16]:
def generate_valid_moves(
    board: Board,
    current_player: breakthrough.Player
) -> list[Move]:
    """
    Generates a list containing all possible moves in a particular position for the current player
    to move. Return an empty list if there are no possible moves.

    Parameters
    ----------
    board: 2D list of lists. Contains characters "B", "W", and "_",
        representing black pawn, white pawn, and empty cell, respectively.

    current_player: breakthrough.Player, the colour of the current player to move.

    Returns
    -------
    A list of Moves.
    """
    """ YOUR CODE HERE """
    #enumerate row and col
    #iterate over -1 to 2 for row and col 
    #check if move is valid, append to list
    moves = []
    for r,row in enumerate(board):
        for c,tile in enumerate(row):
            for i in range(-1,2):
                for j in range(-1,2):
                    if breakthrough.is_valid_move(board,(r,c),(r+i,c+j),current_player):
                        moves.append(((r,c),(r+i,c+j)))
    return moves
    raise NotImplementedError
    """ YOUR CODE END HERE """

# Test cases
board_21 = [
    list("__B___"),
    list("______"),
    list("______"),
    list("______"),
    list("______"),
    list("_W____"),
]

board_22 = [
    list("____B_"),
    list("_____B"),
    list("_W____"),
    list("__W___"),
    list("______"),
    list("______"),
]

board_23 = [
    list("______"),
    list("__B___"),
    list("_WWW__"),
    list("______"),
    list("______"),
    list("______"),
]

assert sorted(generate_valid_moves(board_21, breakthrough.Player.BLACK)) == [((0, 2), (1, 1)), ((0, 2), (1, 2)), ((0, 2), (1, 3))], "Moves generated for black on board_21 is incorrect"
assert sorted(generate_valid_moves(board_21, breakthrough.Player.WHITE)) == [((5, 1), (4, 0)), ((5, 1), (4, 1)), ((5, 1), (4, 2))], "Moves generated for white on board_21 is incorrect"
assert sorted(generate_valid_moves(board_22, breakthrough.Player.BLACK)) == [((0, 4), (1, 3)), ((0, 4), (1, 4)), ((1, 5), (2, 4)), ((1, 5), (2, 5))], "Moves generated for black on board_22 is incorrect"
assert sorted(generate_valid_moves(board_22, breakthrough.Player.WHITE)) == [((2, 1), (1, 0)), ((2, 1), (1, 1)), ((2, 1), (1, 2)), ((3, 2), (2, 2)), ((3, 2), (2, 3))], "Moves generated for white on board_22 is incorrect"
assert sorted(generate_valid_moves(board_23, breakthrough.Player.BLACK)) == [((1, 2), (2, 1)), ((1, 2), (2, 3))], "Moves generated for black on board_23 is incorrect"
assert sorted(generate_valid_moves(board_23, breakthrough.Player.WHITE)) == [((2, 1), (1, 0)), ((2, 1), (1, 1)), ((2, 1), (1, 2)), ((2, 2), (1, 1)), ((2, 2), (1, 3)), ((2, 3), (1, 2)), ((2, 3), (1, 3)), ((2, 3), (1, 4))], "Moves generated for white on board_23 is incorrect"

## Minimax Algorithm



Your agent must be able to calculate the game state a few moves in advance, by implementing the **minimax** algorithm.

### Task 3.1: Implement minimax

In the lecture, you have seen the minimax algorithm without and with cutoff. We will implement a minimax algorithm with cutoff in this case, as the depth of the game and the branching factor in certain positions can be (very) large and we do not have the computational power to compute the entire game until the terminal states.

Your minimax function should explore different game states, until either the depth is `max_depth`, or there is a winner. In these cases, your minimax algorithm should use the provided heuristic function, `evaluate(board)` to evaluate the position.

Your minimax function must be able to handle making the first move for either black or white. Regardless of which player moves first, the minimax evaluation should always be done from the perspective of black.

**Note**: For tasks 3.1 to 4.2, if you are certain that your solution is correct but the test cases fail on Coursemology due to timeout, just rerun your code. Depending on the load on Coursemology, a correct solution might still timeout.

In [17]:

def minimax(
    board: Board,
    depth: int,
    max_depth: int,
    current_player: breakthrough.Player,
    generate_valid_moves: Callable
) -> tuple[Score, Move]:
    """
    Finds the best move for the current player and corresponding evaluation from black's
    perspective for the input board state. Return breakthrough.MOVE_NONE if no move is possible
    (e.g. when the game is over).

    Parameters
    ----------
    board: 2D list of lists. Contains characters "B", "W", and "_",
        representing black pawn, white pawn, and empty cell, respectively. Your function may modify
        the board internally, but the original board passed as an argument must remain unchanged.

    depth: int, the depth to search for the best move. When this is equal to `max_depth`, you
        should get the evaluation of the position using the provided heuristic function.

    max_depth: int, the maximum depth for cutoff.

    current_player: breakthrough.Player, the colour of the current player to move.

    generate_valid_moves: Callable, move generation function. Will be provided on Coursemology.

    Returns
    -------
    A tuple (evaluation, ((src_row, src_col), (dst_row, dst_col))):
    evaluation: the best score that the current player to move can achieve.
    src_row, src_col: position of the pawn to move.
    dst_row, dst_col: position to move the pawn to.
    """
    """ YOUR CODE HERE """
    def is_terminal(board,depth,max_depth,evaluate): #check if game is over based on depth
        if depth == 0 and abs(evaluate(board)) == breakthrough.WIN: #check initial conditions if game is over
            return True , (evaluate(board),breakthrough.MOVE_NONE)
        elif depth == max_depth or breakthrough.is_game_over(board):
            return True , (evaluate(board),breakthrough.MOVE_NONE)
        else:
            return False , None

    def max_value(board,depth,current_player):  
        terminal, value = is_terminal(board,depth,max_depth,evaluate)
        if terminal:
            return value
        #initialize score and move
        Score = -breakthrough.INF
        Move = breakthrough.MOVE_NONE
        for move in generate_valid_moves(board,current_player):
            src,dst = move
            new_board = breakthrough.make_move(board,src,dst,current_player,False)
            new_v  = min_value(new_board,depth+1,current_player.get_opponent())[0]
            if new_v > Score:
                Score = new_v
                Move = move
        return (Score,Move)

    def min_value(board,depth,current_player):
        terminal, value = is_terminal(board,depth,max_depth,evaluate)
        if terminal:
            return value
        Score = breakthrough.INF
        Move = breakthrough.MOVE_NONE
        for move in generate_valid_moves(board,current_player):
            src,dst = move
            new_board = breakthrough.make_move(board,src,dst,current_player,False)
            new_v = max_value(new_board,depth+1,current_player.get_opponent())[0]
            if new_v < Score:    
                Score = new_v
                Move = move
        return (Score,Move)

    #white is for min, black is for max
    if current_player == breakthrough.Player.BLACK:
        return max_value(board,depth,current_player)
    else:
        return min_value(board,depth,current_player)
    raise NotImplementedError
    """ YOUR CODE END HERE """

In [18]:
preservation_board = [
    list("___B__"),
    list("______"),
    list("_B__B_"),
    list("W____W"),
    list("___W__"),
    list("_W____"),
]

game_over_board_1 = [
    list("______"),
    list("_W____"),
    list("______"),
    list("______"),
    list("______"),
    list("______"),
]

game_over_board_2 = [
    list("______"),
    list("______"),
    list("______"),
    list("______"),
    list("_B____"),
    list("______"),
]

max_depth_board_1 = [
    list("______"),
    list("W_____"),
    list("______"),
    list("_____B"),
    list("______"),
    list("______"),
]

max_depth_board_2 = [
    list("______"),
    list("______"),
    list("W_____"),
    list("______"),
    list("_____B"),
    list("______"),
]

player_switching_board = [
    list("___B__"),
    list("______"),
    list("_B__B_"),
    list("W____W"),
    list("______"),
    list("___W__"),
]

board_31 = [
    list("______"),
    list("___B__"),
    list("____BB"),
    list("___WB_"),
    list("_B__WW"),
    list("_WW___"),
]

board_32 = [
    list("______"),
    list("_____B"),
    list("_W____"),
    list("______"),
    list("______"),
    list("______"),
]

board_33 = [
    list("______"),
    list("__B___"),
    list("W_____"),
    list("___B_B"),
    list("__W___"),
    list("______"),
]

board_34 = [
    list("______"),
    list("____BB"),
    list("__B___"),
    list("_W_B_B"),
    list("__W_W_"),
    list("___WW_"),
]

board_35 = [
    list("______"),
    list("______"),
    list("______"),
    list("_____W"),
    list("_B____"),
    list("______"),
]

def invoke_search_fn(search_fn, board, max_depth, current_player):
    if "alpha_beta" in search_fn.__name__:
        return search_fn(board, 0, max_depth, -breakthrough.INF, breakthrough.INF, current_player, generate_valid_moves)
    else:
        return search_fn(board, 0, max_depth, current_player, generate_valid_moves)

def test_board_preservation(search_fn):
    control_board = copy.deepcopy(preservation_board)
    input_board = copy.deepcopy(preservation_board)
    invoke_search_fn(search_fn, input_board, 2, breakthrough.Player.BLACK)
    assert control_board == input_board, "Your function may be modifying the board by making moves in place."

def test_game_over(search_fn, board, expected_score):
    score, move = invoke_search_fn(search_fn, board, 1, breakthrough.Player.BLACK)
    assert score == expected_score, "Your function might not have terminated when the game is over."
    assert move == breakthrough.MOVE_NONE, "Your function might not be returning breakthrough.MOVE_NONE when no moves are possible or it might be generating moves for the opponent instead."

def test_max_depth(search_fn, board, current_player, expected_moves):
    score, move = invoke_search_fn(search_fn, board, 1, current_player)
    assert score == 0, f"Your function may not be terminating at max depth or you may be initialising {current_player.value}'s score incorrectly."
    assert move in expected_moves, f"Your function does not move {current_player.value} pieces during {current_player.value}'s turn, or initialises {current_player.value}'s score incorrectly or terminates early."

def test_player_switching(search_fn, current_player):
    score, _ = invoke_search_fn(search_fn, player_switching_board, 2, current_player)
    assert score != 2, "Your function may not be switching player's colours correctly after making a move."
    assert score == 0, "Your function may not be making the most optimal move at each depth."

def test_search(search_fn, board, max_depth, current_player, expected_score, expected_moves):
    score, move = invoke_search_fn(search_fn, board, max_depth, current_player)
    assert score == expected_score, f"Final evaluation score should be {expected_score} instead of {score}."
    assert move in expected_moves, f"Your function does not correctly move {current_player.value}'s pieces despite having the correct evaluation."

In [19]:
# Test cases
test_board_preservation(minimax)
test_game_over(minimax, game_over_board_1, -breakthrough.WIN)
test_game_over(minimax, game_over_board_2, breakthrough.WIN)
test_max_depth(minimax, max_depth_board_1, breakthrough.Player.BLACK, [((3, 5), (4, 4)), ((3, 5), (4, 5))])
test_max_depth(minimax, max_depth_board_2, breakthrough.Player.WHITE, [((2, 0), (1, 0)), ((2, 0), (1, 1))])
test_player_switching(minimax, breakthrough.Player.BLACK)
test_player_switching(minimax, breakthrough.Player.WHITE)

test_search(minimax, board_31, 1, breakthrough.Player.BLACK, breakthrough.WIN, [((4, 1), (5, 0)), ((4, 1), (5, 2))])
test_search(minimax, board_32, 4, breakthrough.Player.BLACK, -breakthrough.WIN, [((1, 5), (2, 5)), ((1, 5), (2, 4))])
test_search(minimax, board_33, 3, breakthrough.Player.WHITE, -breakthrough.WIN, [((2, 0), (1, 0)), ((2, 0), (1, 1))])
test_search(minimax, board_34, 3, breakthrough.Player.WHITE, -1, [((3, 1), (2, 2)), ((4, 2), (3, 3)), ((4, 4), (3, 3)), ((4, 4), (3, 5))])
# test_search(minimax, board_35, 4, breakthrough.Player.WHITE, breakthrough.WIN,  [((1, 5), (2, 5)), ((1, 5), (2, 4))])

### Task 3.2: Implement negamax

You may notice that Breakthrough is a zero-sum game. It means that the sum of the evaluation scores of the two players should be zero.

For example, we can consider a position in which there are _9 black pawns_ and _6 white pawns_. Using the sample heuristic function given at the start:

- The evaluation score of black in this position is `+3`.
- The evaluation score of white in this position is `-3`.

Using this property, we can simplify the implementation of minimax. Instead of taking the maximum and minimum scores for black and white respectively, we can negate the score of the opposite player and take the maximum score. This version is called **negamax**. Your negamax function must be able to handle making the first move for either black or white.

In [20]:
def negamax(
    board: Board,
    depth: int,
    max_depth: int,
    current_player: breakthrough.Player,
    generate_valid_moves: Callable
) -> tuple[Score, Move]:
    """
    Finds the best move for the current player and corresponding evaluation for the input board
    state. Return breakthrough.MOVE_NONE if no move is possible (e.g. when the game is over).

    Parameters
    ----------
    board: 2D list of lists. Contains characters "B", "W", and "_",
        representing black pawn, white pawn, and empty cell, respectively. Your function may modify
        the board internally, but the original board passed as an argument must remain unchanged.

    depth: int, the depth to search for the best move. When this is equal to `max_depth`, you
        should get the evaluation of the position using the provided heuristic function.

    max_depth: int, the maximum depth for cutoff.

    current_player: breakthrough.Player, the colour of the current player to move.

    generate_valid_moves: Callable, move generation function. Will be provided on Coursemology.

    Returns
    -------
    A tuple (evaluation, ((src_row, src_col), (dst_row, dst_col))):
    evaluation: the best score that the current player to move can achieve.
    src_row, src_col: position of the pawn to move.
    dst_row, dst_col: position to move the pawn to.
    """
    """ YOUR CODE HERE """
    def is_terminal(board,depth,max_depth,evaluate): #check if game is over based on depth
        if depth == 0 and abs(evaluate(board)) == breakthrough.WIN:
            return True , (evaluate(board),breakthrough.MOVE_NONE)
        elif breakthrough.is_game_over(board): 
            return True , (-abs(evaluate(board)),breakthrough.MOVE_NONE) #return negative value for winning we have a negation in the negamax function (line 191 ) -> always want win to be maximum positive value for both min and max
        elif depth == max_depth:
            return True , (evaluate(board),breakthrough.MOVE_NONE)
        
        else:
            return False , None         

    terminal, value = is_terminal(board,depth,max_depth,evaluate)
    if terminal:
        return value    
    Score = -breakthrough.INF
    for move in generate_valid_moves(board,current_player):
        src,dst = move
        new_board = breakthrough.make_move(board,src,dst,current_player,False)
        # if breakthrough.is_game_over(new_board): #this means we need winning to always be positive and not aff
        #         new_v =  breakthrough.WIN
        # else:
        new_v = -negamax(new_board,depth+1,max_depth,current_player.get_opponent(),generate_valid_moves)[0]
        if new_v > Score:
            Score = new_v
            Move = move
    # print(Score)
    if depth == 0 and current_player == breakthrough.Player.WHITE:
        Score = -Score
    return (Score,Move)
    raise NotImplementedError
    """ YOUR CODE END HERE """

In [21]:
# # Test cases
test_board_preservation(negamax)
test_game_over(negamax, game_over_board_1, -breakthrough.WIN)
test_game_over(negamax, game_over_board_2, breakthrough.WIN)
test_max_depth(negamax, max_depth_board_1, breakthrough.Player.BLACK, [((3, 5), (4, 4)), ((3, 5), (4, 5))])
test_max_depth(negamax, max_depth_board_2, breakthrough.Player.WHITE, [((2, 0), (1, 0)), ((2, 0), (1, 1))])
test_player_switching(negamax, breakthrough.Player.BLACK)
test_player_switching(negamax, breakthrough.Player.WHITE)

test_search(negamax, board_31, 1, breakthrough.Player.BLACK, breakthrough.WIN, [((4, 1), (5, 0)), ((4, 1), (5, 2))])
test_search(negamax, board_32, 4, breakthrough.Player.BLACK, -breakthrough.WIN, [((1, 5), (2, 5)), ((1, 5), (2, 4))])
test_search(negamax, board_33, 3, breakthrough.Player.WHITE, -breakthrough.WIN, [((2, 0), (1, 0)), ((2, 0), (1, 1))])
test_search(negamax, board_34, 3, breakthrough.Player.WHITE, -1, [((3, 1), (2, 2)), ((4, 2), (3, 3)), ((4, 4), (3, 3)), ((4, 4), (3, 5))])

## Alpha-beta Pruning

With minimax (or negamax), our agent can see the future within a few moves. However, the naive implementation of minimax (or negamax) may explore many redundant states, which slows down our agent. As discussed in the lecture, we can apply **alpha-beta pruning** to eliminate unnecessary states, thereby improving our agent's speed and its ability to see even further into the future. This will increase our agent's strength and its likelihood of winning the game. 

First, you should try to integrate alpha-beta pruning with the standard minimax algorithm. Similar to Task 3.1, your minimax function with alpha-beta pruning must be able to handle making the first move for either black or white.

### Task 4.1: Integrate alpha-beta pruning into minimax

In [22]:
def minimax_alpha_beta(
    board: Board,
    depth: int,
    max_depth: int,
    alpha: Score,
    beta: Score,
    current_player: breakthrough.Player,
    generate_valid_moves: Callable
) -> tuple[Score, Move]:
    """
    Finds the best move for the current player and corresponding evaluation from black's
    perspective for the input board state. Return breakthrough.MOVE_NONE if no move is possible
    (e.g. when the game is over).

    Parameters
    ----------
    board: 2D list of lists. Contains characters "B", "W", and "_",
        representing black pawn, white pawn, and empty cell, respectively. Your function may modify
        the board internally, but the original board passed as an argument must remain unchanged.

    depth: int, the depth to search for the best move. When this is equal to `max_depth`, you
        should get the evaluation of the position using the provided heuristic function.

    max_depth: int, the maximum depth for cutoff.

    alpha: Score. The alpha value in a given state.

    beta: Score. The beta value in a given state.

    current_player: breakthrough.Player, the colour of the current player
        to move.

    generate_valid_moves: Callable, move generation function. Will be
        provided on Coursemology.

    Returns
    -------
    A tuple (evaluation, ((src_row, src_col), (dst_row, dst_col))):
    evaluation: the best score that the current player to move can achieve.
    src_row, src_col: position of the pawn to move.
    dst_row, dst_col: position to move the pawn to.
    """
    """ YOUR CODE HERE """
    def is_terminal(board,depth,max_depth,evaluate): #check if game is over based on depth
        if depth == 0 and abs(evaluate(board)) == breakthrough.WIN: #check initial conditions if game is over
            return True , (evaluate(board),breakthrough.MOVE_NONE)
        elif depth == max_depth or breakthrough.is_game_over(board):
            return True , (evaluate(board),breakthrough.MOVE_NONE)
        else:
            return False , None

    def max_value(board,depth,current_player,alpha,beta):  
        terminal, value = is_terminal(board,depth,max_depth,evaluate)
        if terminal:
            return value
        #initialize score and move
        Score = -breakthrough.INF
        Move = breakthrough.MOVE_NONE
        for move in generate_valid_moves(board,current_player):
            src,dst = move
            new_board = breakthrough.make_move(board,src,dst,current_player,False)
            v, _ = min_value(new_board,depth+1,current_player.get_opponent(),alpha,beta)
            if v > Score:
                Score = v
                Move = move
            alpha = max(Score,alpha)
            if Score >= beta: 
                return (Score,Move) #since we are in max, any value greater than beta will be ignored and any smaller value will be rejected in previous phase                
        return (Score,Move)

    def min_value(board,depth,current_player,alpha,beta):
        terminal, value = is_terminal(board,depth,max_depth,evaluate)
        if terminal:
            return value
        Score = breakthrough.INF
        Move = breakthrough.MOVE_NONE
        for move in generate_valid_moves(board,current_player):
            src,dst = move
            new_board = breakthrough.make_move(board,src,dst,current_player,False)
            v,_ = max_value(new_board,depth+1,current_player.get_opponent(),alpha,beta)
            if v < Score:
                Score = v
                Move = move
            beta = min(Score,beta)
            if Score <= alpha: #note it cannot be beta <= alpha  as we might not have explored all moves and doing so will instantly prune unexplored trees espcially when they have score higher than alpha
                return (Score,Move) #since we are in min, any value smaller than alpha will be ignored and any greater value will be rejected in previous phase
        return (Score,Move)

    #white is for min, black is for max
    #note alpha is for max = -inf , beta is for min = inf
    if current_player == breakthrough.Player.BLACK:
        return max_value(board,depth,current_player,-breakthrough.INF,breakthrough.INF)
    else:
        return min_value(board,depth,current_player,-breakthrough.INF,breakthrough.INF)
    raise NotImplementedError
    """ YOUR CODE END HERE """

In [23]:
board_41 = [
    list("______"),
    list("__BB__"),
    list("____BB"),
    list("WBW_B_"),
    list("____WW"),
    list("_WW___"),
]

board_42 = [
    list("____B_"),
    list("__BB__"),
    list("______"),
    list("_WWW__"),
    list("____W_"),
    list("______"),
]

In [24]:
# Test cases
test_board_preservation(minimax_alpha_beta)
test_game_over(minimax_alpha_beta, game_over_board_1, -breakthrough.WIN)
test_game_over(minimax_alpha_beta, game_over_board_2, breakthrough.WIN)
test_max_depth(minimax_alpha_beta, max_depth_board_1, breakthrough.Player.BLACK, [((3, 5), (4, 4)), ((3, 5), (4, 5))])
test_max_depth(minimax_alpha_beta, max_depth_board_2, breakthrough.Player.WHITE, [((2, 0), (1, 0)), ((2, 0), (1, 1))])
test_player_switching(minimax_alpha_beta, breakthrough.Player.BLACK)
test_player_switching(minimax_alpha_beta, breakthrough.Player.WHITE)

test_search(minimax_alpha_beta, board_41, 3, breakthrough.Player.BLACK, breakthrough.WIN, [((3, 4), (4, 5))])
test_search(minimax_alpha_beta, board_42, 6, breakthrough.Player.BLACK, -breakthrough.WIN, [((0, 4), (1, 4)), ((0, 4), (1, 5)), ((1, 2), (2, 2)), ((1, 2), (2, 1)), ((1, 2), (2, 3)), ((1, 3), (2, 3)), ((1, 3), (2, 2)), ((1, 3), (2, 4))])
test_search(minimax_alpha_beta, board_33, 3, breakthrough.Player.WHITE, -breakthrough.WIN, [((2, 0), (1, 0)), ((2, 0), (1, 1))])
test_search(minimax_alpha_beta, board_34, 3, breakthrough.Player.WHITE, -1, [((3, 1), (2, 2)), ((4, 2), (3, 3)), ((4, 4), (3, 3)), ((4, 4), (3, 5))])

### Task 4.2: Integrate alpha-beta pruning into negamax

At this stage, you may wonder: why don't we integrate alpha-beta pruning with the alternative, negamax? Of course, we can also incorporate alpha-beta pruning into our negamax algorithm to improve its performance.

Remember, we exploited the zero-sum property of the game to implement negamax by negating the evaluation score, and always taking the maximum score instead of alternating between the maximum and minimum. You need to further exploit this property to correctly integrate alpha-beta pruning into negamax. To help you, these are some questions that you can try to answer:

- What is the meaning of `alpha` and `beta`?
- From the perspective of the opponent, what is the corresponding `alpha` and `beta`? Can I exploit the zero-sum property here?

In [25]:
def negamax_alpha_beta(
    board: Board,
    depth: int,
    max_depth: int,
    alpha: Score,
    beta: Score,
    current_player: breakthrough.Player,
    generate_valid_moves: Callable
) -> tuple[Score, Move]:
    """
    Finds the best move for the current player and corresponding evaluation for the input board
    state. Return breakthrough.MOVE_NONE if no move is possible (e.g. when the game is over).

    Parameters
    ----------
    board: 2D list of lists. Contains characters "B", "W", and "_",
        representing black pawn, white pawn, and empty cell, respectively. Your function may modify
        the board internally, but the original board passed as an argument must remain unchanged.

    depth: int, the depth to search for the best move. When this is equal to `max_depth`, you
        should get the evaluation of the position using the provided heuristic function.

    max_depth: int, the maximum depth for cutoff.

    alpha: Score. The alpha value in a given state.

    beta: Score. The beta value in a given state.

    current_player: breakthrough.Player, the colour of the current player to move.

    generate_valid_moves: Callable, move generation function. Will be provided on Coursemology.

    Returns
    -------
    A tuple (evaluation, ((src_row, src_col), (dst_row, dst_col))):
    evaluation: the best score that the current player to move can achieve.
    src_row, src_col: position of the pawn to move.
    dst_row, dst_col: position to move the pawn to.
    """
    """ YOUR CODE HERE """
    
    """
    prunning conditions
    we need to keep rotating alpha and beta (since all values are max only alpha will change since its -inf, therefore we at each negamax call, we need to rotate alpha and beta)
    when rotating between alpha and beta, we need to negate the value of the score (ensure that when recursing to root, alpha = -inf and beta = inf)
    therefore,
    since alpha is always being updated at current, alpha at previous needs to be somehow updated to beta when at current
    therefore, we have to swap alpha and beta at each negamax call and negate them
    parent -> alpha = alpha , beta = beta
    leaf node -> alpha = -beta , beta = -alpha
    parent node -> alpha = -beta , beta = -alpha (revert to original config)
    other leaf node -> alpha = -beta , beta = -alpha
    
    """
    def is_terminal(board,depth,max_depth,evaluate):
        if depth == 0 and abs(evaluate(board)) == breakthrough.WIN:
            return True , (evaluate(board),breakthrough.MOVE_NONE)
        elif breakthrough.is_game_over(board):
            return True , (-abs(evaluate(board)),breakthrough.MOVE_NONE)
        elif depth == max_depth:
            return True , (evaluate(board),breakthrough.MOVE_NONE)
        return False , None
    
    terminal, value = is_terminal(board,depth,max_depth,evaluate)
    if terminal:
        return value
    Score = -breakthrough.INF
    for move in generate_valid_moves(board,current_player):
        src,dst = move
        new_board = breakthrough.make_move(board,src,dst,current_player,False)
        v = -negamax_alpha_beta(new_board,depth+1,max_depth,-beta,-alpha,current_player.get_opponent(),generate_valid_moves)[0]
        if v > Score:
            Score = v
            Move = move
        alpha = max(Score,alpha)
        if Score >= beta:
            return (Score,Move)

    if depth == 0 and current_player == breakthrough.Player.WHITE:
        Score = -Score
    return (Score,Move)
    raise NotImplementedError
    """ YOUR CODE END HERE """

In [26]:
# Test cases
test_board_preservation(negamax_alpha_beta)
test_game_over(negamax_alpha_beta, game_over_board_1, -breakthrough.WIN)
test_game_over(negamax_alpha_beta, game_over_board_2, breakthrough.WIN)
test_max_depth(negamax_alpha_beta, max_depth_board_1, breakthrough.Player.BLACK, [((3, 5), (4, 4)), ((3, 5), (4, 5))])
test_max_depth(negamax_alpha_beta, max_depth_board_2, breakthrough.Player.WHITE, [((2, 0), (1, 0)), ((2, 0), (1, 1))])
test_player_switching(negamax_alpha_beta, breakthrough.Player.BLACK)
test_player_switching(negamax_alpha_beta, breakthrough.Player.WHITE)

test_search(negamax_alpha_beta, board_41, 3, breakthrough.Player.BLACK, breakthrough.WIN, [((3, 4), (4, 5))])
test_search(negamax_alpha_beta, board_42, 6, breakthrough.Player.BLACK, -breakthrough.WIN, [((0, 4), (1, 4)), ((0, 4), (1, 5)), ((1, 2), (2, 2)), ((1, 2), (2, 1)), ((1, 2), (2, 3)), ((1, 3), (2, 3)), ((1, 3), (2, 2)), ((1, 3), (2, 4))])
test_search(negamax_alpha_beta, board_33, 3, breakthrough.Player.WHITE, -breakthrough.WIN, [((2, 0), (1, 0)), ((2, 0), (1, 1))])
test_search(negamax_alpha_beta, board_34, 3, breakthrough.Player.WHITE, -1, [((3, 1), (2, 2)), ((4, 2), (3, 3)), ((4, 4), (3, 3)), ((4, 4), (3, 5))])

## Heuristic Function

Phew, we finish the search algorithm! But, our heuristic function is too simple - it may not give the best evaluation for a position and we need a better one. Therefore, you shall implement the improved heuristic function described below.

### Task 5.1: Implement an improved heuristic function

Recall that the heuristic function should return a larger value when black is closer to winning. If black is closer to winning, black should have more pieces closer to row 5 compared to white having pieces closer to row 0. Of course this is not necessarily the case since you only need one piece to make it through while the rest remain behind, but this is just a heuristic after all. Thus, in this heuristic you are about to implement, we add more points if a black piece is closer to the end, and subtract more points if a white piece is closer to the end. The exact amount of points is shown in the figures below.

<pre>
<p style="text-align: center;">
<img src="images/black_heuristic.png", width = 300>
Figure 4. Points to add for each black piece in the square
<p style="text-align: center;">
<img src="images/white_heuristic.png", width = 300>
Figure 5. Points to subtract for each white piece in the square
</p>
</pre>

For example, if there are two black pieces on (0, 4) and (3, 2), and a white piece on (1, 4), then the output of the heuristic function should be `10 + 40 - 50 = 0`. Additionally, return `breakthrough.WIN` if any of black's pieces reach the end, and `-breakthrough.WIN` if any of white's pieces reach the end. Similarly, if white has no pieces, return `breakthrough.WIN`, and if black has no pieces, return `-breakthrough.WIN`. The value of `breakthrough.WIN` can be found in `breakthrough.py` and has a value of `21092109`.

In [ ]:

def improved_evaluate(board: Board) -> Score:
    """
    Returns the score of the current position with an improved heuristic.

    Parameters
    ----------
    board: 2D list of lists. Contains characters "B", "W", and "_",
        representing black pawn, white pawn, and empty cell, respectively.

    Returns
    -------
    An improved evaluation (as a Score).
    """
    """ YOUR CODE HERE """
    h_n = 0
    heuristic = [10,10,20,40,50]
    b_count = 0
    w_count = 0
    for r,row in enumerate(board):
        for tile in row:
            if tile == "B":
                if r == 5:
                    return breakthrough.WIN
                h_n += heuristic[r]
                b_count += 1
            elif tile == "W":
                if r == 0:
                    return -breakthrough.WIN
                h_n -= heuristic[5-r]
                w_count += 1
    if w_count == 0: #no more white pieces, b win
        return breakthrough.WIN
    if b_count == 0: #no more black pieces, white win
        return -breakthrough.WIN
    return  h_n
    """ YOUR CODE END HERE """

# Test cases
board_51 = [
    list("___B__"),
    list("___W__"),
    list("______"),
    list("__B___"),
    list("______"),
    list("______"),
]

board_52 = [
    list("___BW_"),
    list("___W__"),
    list("______"),
    list("______"),
    list("______"),
    list("______"),
]

board_53 = [
    list("______"),
    list("______"),
    list("______"),
    list("__B___"),
    list("______"),
    list("______"),
]

assert improved_evaluate(board_51) == 0, "Your improved evaluation function should return 0 for this board. But instead returned: " + str(improved_evaluate(board_51))
assert improved_evaluate(board_52) == -breakthrough.WIN, "Your improved evaluation function does not correctly evaluate won positions."
assert improved_evaluate(board_53) == breakthrough.WIN, "Your improved evaluation function does not correctly evaluate won positions." + str(improved_evaluate(board_53))

## Test cases

To assist with your implementation, we have provided some examples as test cases. These are not sufficient to ensure that your code works correctly, and we encourage you to write your own additional test cases to test and debug your code.

Note that your answers may be slightly different from the answers provided since multiple valid solutions may exist. During grading, your code will be evaluated on hidden test cases on top of the ones we have provided to check the quality of your search functions and algorithms.